![logo](https://github.com/cristiandarioortegayubro/BA/blob/main/dba.png?raw=true)

# **Trabajo Final - Módulo 11 - Aprendizaje Automático - Machine Learning - Clustering**

![logo](https://www.python.org/static/community_logos/python-powered-w-100x40.png)


## ***Gómez Ocampo Matías*** 


# **Carga de módulos y datos necesarios**

In [1]:
import pandas as pd #Para el tratamiento de los datos
import numpy as np

In [2]:
from sklearn.preprocessing import StandardScaler #Para el preprocesamiento y modelo
from sklearn.cluster import KMeans

In [3]:
from sklearn.decomposition import PCA #Para Análisis de Componentes Principales

In [4]:
import plotly.express as px #para gráficos
import plotly.graph_objects as go

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# **Creación del DataFrame**

In [5]:
datos = "https://raw.githubusercontent.com/LucaAPiattelli/Diplomatura_Business_Analytics_UDA/main/Modulo_11_Agrupacion/Mall_Customers.csv"
df = pd.read_csv(datos)
df

,CustomerID,Gender,Age,AnnualIncome,SpendingScore
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40
...,...,...,...,...,...
195,196,Female,35,120,79
196,197,Female,45,126,28
197,198,Male,32,126,74
198,199,Male,32,137,18


- CustomerID - ID de cliente
- Gender - Genero
- Age - Edad
- AnualIncome - Ingreso Anual
- SpendingScore - Score de gasto en el mall

# **EDA**

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CustomerID     200 non-null    int64 
 1   Gender         200 non-null    object
 2   Age            200 non-null    int64 
 3   AnnualIncome   200 non-null    int64 
 4   SpendingScore  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB


In [7]:
df.head(5)

,CustomerID,Gender,Age,AnnualIncome,SpendingScore
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [8]:
df.shape

(200, 5)

In [9]:
round(df.describe(),2).T #visualizamos los datos estadísdicos del df

,count,mean,std,min,25%,50%,75%,max
CustomerID,200.0,100.50,57.88,1.0,50.75,100.5,150.25,200.0
Age,200.0,38.85,13.97,18.0,28.75,36.0,49.00,70.0
AnnualIncome,200.0,60.56,26.26,15.0,41.50,61.5,78.00,137.0
SpendingScore,200.0,50.20,25.82,1.0,34.75,50.0,73.00,99.0


In [10]:
df.isnull().sum().sort_values(ascending=False) #no hay nulos aparentemente

CustomerID       0
Gender           0
Age              0
AnnualIncome     0
SpendingScore    0
dtype: int64

In [11]:
df1 = df.copy() #hago una copia del dataframe antes de dropear la columna que no es necesaria para el análisis

In [12]:
df1.drop(columns='CustomerID', inplace = True) #dropeo la columna CustomerID

In [13]:
df1.head(3)

,Gender,Age,AnnualIncome,SpendingScore
0,Male,19,15,39
1,Male,21,15,81
2,Female,20,16,6


In [14]:
df  = df.rename(columns={#renombrar columnas
     'Gender':'genero',
     'Age':'edad',
     'AnnualIncome':'ingreso_anual',
     'SpendingScore':'importe_gastado',
     }) 

In [15]:
df1 = df1.rename(columns={#renombrar columnas
     'Gender':'genero',
     'Age':'edad',
     'AnnualIncome':'ingreso_anual',
     'SpendingScore':'importe_gastado',
     }) 

In [16]:
df1.head(3)

,genero,edad,ingreso_anual,importe_gastado
0,Male,19,15,39
1,Male,21,15,81
2,Female,20,16,6


# **Generación del modelo de agrupación**

# Normalización de los datos

In [17]:
df1 = pd.get_dummies(df, drop_first = True ) #convertimos las variables categóricas a numéricas 

In [18]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df1)
df.shape == df_scaled.shape

True

# Análisis de componentes principales

In [19]:
pca = PCA(n_components = 3).fit(df_scaled)
pca_values = pca.transform(df_scaled)

In [20]:
pca_values = pd.DataFrame(data = pca_values, columns=['PC1','PC2',"PC3"])

In [21]:
df1.shape[0] == pca_values.shape[0]

True

In [22]:
pca_values.shape

(200, 3)

# Número de clústers

In [23]:
n_clusters = 15
elbow = []
columnas = ["cluster_range", "inertia"]

In [24]:
for i in range(1, n_clusters):
    kmeans = KMeans(i)
    kmeans.fit(pca_values)
    elbow.append([i,kmeans.inertia_])

In [25]:
clusters = pd.DataFrame(elbow)
clusters.columns = columnas

# Visualización del método Elbow

In [26]:
fig = px.line(x = pd.DataFrame(elbow)[0],
              y = pd.DataFrame(elbow)[1],
              markers = True,
              title = "Metodo del codo",
              template = "simple_white",
              labels = {"cluster_range":"clusters"})
fig.show()

# Modelo K-Means

In [27]:
kmeans = KMeans(4) #la cantidad de clústers la determinamos en base las obervaciones hechas en el método anterior aproximadamente 4 clústers

In [28]:
kmeans.fit(pca_values)

KMeans(n_clusters=4)

# Visualización del modelo

In [29]:
Scene = dict(xaxis = dict(title  = 'PC1'), yaxis = dict(title  = 'PC2'), zaxis = dict(title  = 'PC3'))

In [30]:
labels = kmeans.labels_

In [31]:
trace = go.Scatter3d(x = pca_values['PC1'], 
                     y = pca_values['PC2'], 
                     z = pca_values['PC3'], 
                     mode = 'markers', 
                     marker = dict(color = labels, size= 10))

In [32]:
layout = go.Layout(margin = dict(l = 0,r = 0),
                   scene = Scene,
                   height = 800,
                   width = 800)

In [33]:
data = [trace]

In [34]:
fig = go.Figure(data = data, layout = layout)

In [35]:
fig.show()

In [36]:
pca_values['clusters'] = labels

In [37]:
round(pca_values,3)

,PC1,PC2,PC3,clusters
0,-2.317,-0.454,1.454,1
1,-2.272,-1.474,1.713,1
2,-2.454,-0.007,-0.734,0
3,-2.386,-1.750,-0.291,1
4,-2.397,-0.369,-0.637,1
...,...,...,...,...
195,2.728,-1.116,-0.856,2
196,2.835,0.739,-1.330,2
197,3.073,-0.645,1.079,3
198,3.332,0.854,0.675,3


In [38]:
df1['clusters'] = labels

In [39]:
df1.head()

,CustomerID,edad,ingreso_anual,importe_gastado,genero_Male,clusters
0,1,19,15,39,1,1
1,2,21,15,81,1,1
2,3,20,16,6,0,0
3,4,23,16,77,0,1
4,5,31,17,40,0,1


In [40]:
df['clusters'] = labels

In [41]:
df.head()

,CustomerID,genero,edad,ingreso_anual,importe_gastado,clusters
0,1,Male,19,15,39,1
1,2,Male,21,15,81,1
2,3,Female,20,16,6,0
3,4,Female,23,16,77,1
4,5,Female,31,17,40,1


In [42]:
df1.shape[0] == df.shape[0]

True

In [43]:
df.drop(columns='CustomerID', inplace = True) #dropeo la columna CustomerID del dataframe original dado que no me sirve para el análisis de
                                              #los clústers con los histogramas

# Análisis clúster by clúster

In [44]:
for i in df:
  fig = px.histogram(df, 
                     x = i, 
                     facet_col = "clusters",
                     title = i,
                     template = "gridon",
                     color = "clusters")
  fig.show()

# **Conclusiones**

Está difícil la conclusión dado que hay muchos datos que son comunes en todos los clústers pero bueno, veamos que sale.

Los clústers con más clientes son el clúster 0, el 2 y el 3. El menos populoso sería el clúster 1.

***Clúster 0***, segmento compuesto principalmente por mujeres con ingresos anuales hasta 75k, pero muy concentrado en 50k, con una restricción presupuestaria marcada en 60k y con un rango etario principalmente formado por gente entre los 30 y los 70 años. 
El nivel de gastos está muy cerca del nivel de ingresos, por lo que entiendo es gente que no puede consumir lujos o cosas de segmentos ABC1, por lo que me enfocaría más en meterles una tarjeta de crédito, como hacía CMR en la década del 90, para que consuman más y me enfocaría en un consumo más de retail.

***Clúster 1*** es el más pequeño de los segmentos, el segmento de menor ingreso y mayor gastos probablamente encontremos muchos 'financieramente incorrectos' en este segmento, sobre apalancados en sus ingresos con un gasto superior a sus ingresos o la alternativa que no le gustaría a AFIP ni a la ATM local, un segmento con un alto nivel de informalidad e ingresos no declarados (ej: monotributista A que se va Qatar a ver el mundial).

***Clúster 2*** es un segmento netamente femenino, con un rango etario amplio menor a 60 años, con un ingreso mayor a los 50k anuales y con un gasto con un alto nivel de variabilidad pero acorde a los ingresos.

***Clúster 3*** son todos masculinos, con una rango etario amplio y con los la concentración de sueldos más altos comparado con los otros segmentos pero con un spending score bastante disperso. Habría que enfocarse en estos hombres adinerados que no tienen una frecuencia normal de gasto y hacer que vengan más seguido a la tienda. Ej: aprovechar la fiebre mundialista y venderles camisetas y figuritas, seguro Pablo se gasta medio sueldo en llenar el albúm y estaría dentro de este segmento.